# Identificando o conservante que leva alguns lotes da ração de uma empresa estragarem  

## Descrição do problema 

Alguns lotes das rações para cachorro de uma empresa têm estragado precocemente e aparentemente um dos conservantes usados pode ser a causa. Na composição da ração são utilizados 4 conservantes, chamados de A, B, C, e D, e as quantidades desses podem variar nos diferentes lotes.  

A base de dados da empresa contém informações sobre as quantidades dos conservantes utilizados em cada lote e indica se o lote estragou ou não. 

O objetivo dessa análise é utilizar Machine Learning para prever os lotes que irão estragar e pelo resultado da modelagem identificar o conservante que está relacionado como o apodrecimento dos lotes.

O algorítmo Random Forest foi utilizado na classificação e o principal componente na determinação do apodrecimento da ração foi o conservante C.

## Carregamento dos dados e pré-analise

In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('tree_dog').getOrCreate()

In [5]:
# carrega os dados
data = spark.read.csv('dog_food.csv', header=True, inferSchema=True)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
# agrupa as variáveis para utilizá-las na classificação
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
# seleciona a coluna com as variáveis preditivas ('features') e a coluna com a variável resposta ('Spoiled')
final_data = output.select('Spoiled', 'features')

In [11]:
final_data.show()

+-------+-------------------+
|Spoiled|           features|
+-------+-------------------+
|    1.0| [4.0,2.0,12.0,3.0]|
|    1.0| [5.0,6.0,12.0,7.0]|
|    1.0| [6.0,2.0,13.0,6.0]|
|    1.0| [4.0,2.0,12.0,1.0]|
|    1.0| [4.0,2.0,12.0,3.0]|
|    1.0|[10.0,3.0,13.0,9.0]|
|    1.0| [8.0,5.0,14.0,5.0]|
|    1.0| [5.0,8.0,12.0,8.0]|
|    1.0| [6.0,5.0,12.0,9.0]|
|    1.0| [3.0,3.0,12.0,1.0]|
|    1.0| [9.0,8.0,11.0,3.0]|
|    1.0|[1.0,10.0,12.0,3.0]|
|    1.0|[1.0,5.0,13.0,10.0]|
|    1.0|[2.0,10.0,12.0,6.0]|
|    1.0|[1.0,10.0,11.0,4.0]|
|    1.0| [5.0,3.0,12.0,2.0]|
|    1.0| [4.0,9.0,11.0,8.0]|
|    1.0| [5.0,1.0,11.0,1.0]|
|    1.0|[4.0,9.0,12.0,10.0]|
|    1.0| [5.0,8.0,10.0,9.0]|
+-------+-------------------+
only showing top 20 rows



## Machine Learning

In [12]:
from pyspark.ml.classification import RandomForestClassifier

In [13]:
# modelo de classificação Random Forest
rf_dog = RandomForestClassifier(labelCol='Spoiled')

In [14]:
# divide os dados entre treino e teste
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [15]:
# ajusta o modelo nos dados de treino
rf_fit = rf_dog.fit(train_data)

In [16]:
# predição nos dados de teste
pred_dog = rf_fit.transform(test_data)

In [17]:
pred_dog.show()

+-------+------------------+--------------------+--------------------+----------+
|Spoiled|          features|       rawPrediction|         probability|prediction|
+-------+------------------+--------------------+--------------------+----------+
|    0.0| [1.0,3.0,9.0,8.0]|[19.8231916982397...|[0.99115958491198...|       0.0|
|    0.0|[1.0,5.0,8.0,10.0]|[18.9795244910941...|[0.94897622455470...|       0.0|
|    0.0| [1.0,6.0,8.0,3.0]|[19.9795244910941...|[0.99897622455470...|       0.0|
|    0.0| [1.0,7.0,8.0,2.0]|[19.9795244910941...|[0.99897622455470...|       0.0|
|    0.0|[1.0,8.0,7.0,10.0]|[19.8947368421052...|[0.99473684210526...|       0.0|
|    0.0| [1.0,8.0,8.0,7.0]|[19.7231368736783...|[0.98615684368391...|       0.0|
|    0.0| [1.0,8.0,8.0,8.0]|[19.7231368736783...|[0.98615684368391...|       0.0|
|    0.0| [1.0,9.0,9.0,7.0]|[19.7231368736783...|[0.98615684368391...|       0.0|
|    0.0| [2.0,1.0,8.0,9.0]|[19.3947916666666...|[0.96973958333333...|       0.0|
|    0.0| [2.0,1

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
my_eval = BinaryClassificationEvaluator(labelCol='Spoiled')

In [21]:
# computa a área sob a curva ROC
AUC = my_eval.evaluate(pred_dog)
AUC

0.9798934108527131

## Determinação da variável de maior importância

In [22]:
# computa as importâncias relativas entre as variáveis
rf_fit.featureImportances

SparseVector(4, {0: 0.0224, 1: 0.02, 2: 0.9325, 3: 0.0252})

O composto responsável por estragar a comida é portanto o composto C. 